***IDS 561 HW4***
Submitted by
Akshaya Sivakumar Karunambika(UIN:671231223)
Manthiramoorthy Cheranthian
(UIN: 668839661)
Rithin Vashishtha
(UIN:676021366)

## Spark installation

In [9]:
from google.colab import drive
drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.mirrors.hoobly.com/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
spark-2.4.5-bin-hadoop2.7/
spark-2.4.5-bin-hadoop2.7/licenses/
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-jtransforms.html
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-zstd.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-zstd-jni.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-xmlenc.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-vis.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-spire.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-sorttable.js.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-slf4j.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-scopt.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-scala.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-sbt-launch-lib.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-respond.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-reflectasm.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-pyrolite.txt
spark-2.4.5-bin-hadoop2.7/licens

# Option 1: Recommender System


In [0]:
# Importing all required libraries
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.sql import Row

Step 1: Import the MovieLens dataset

In [4]:
# Importing the input file
from google.colab import files
uploaded = files.upload()

Saving u.data to u.data


In [11]:
#Read data from u.data file 
from pyspark import SparkConf,SparkContext
from pyspark import SparkContext
from pyspark.sql import functions as func
import pyspark.sql.functions as f
from pyspark.sql import *
from pyspark.sql.functions import col
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import desc
from pyspark import SparkContext
sc =SparkContext.getOrCreate()

import collections
df = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                                       delimiter='\t',\
                               inferschema='true').\
                load("u.data",header=False)
#Renaming the column header                
df2 = df.withColumnRenamed("_c0","userid").withColumnRenamed("_c1","itemid").withColumnRenamed("_c2","rating").withColumnRenamed("_c3","timestamp")
#Displaying the source file
df2.show()

+------+------+------+---------+
|userid|itemid|rating|timestamp|
+------+------+------+---------+
|   196|   242|     3|881250949|
|   186|   302|     3|891717742|
|    22|   377|     1|878887116|
|   244|    51|     2|880606923|
|   166|   346|     1|886397596|
|   298|   474|     4|884182806|
|   115|   265|     2|881171488|
|   253|   465|     5|891628467|
|   305|   451|     3|886324817|
|     6|    86|     3|883603013|
|    62|   257|     2|879372434|
|   286|  1014|     5|879781125|
|   200|   222|     5|876042340|
|   210|    40|     3|891035994|
|   224|    29|     3|888104457|
|   303|   785|     3|879485318|
|   122|   387|     5|879270459|
|   194|   274|     2|879539794|
|   291|  1042|     4|874834944|
|   234|  1184|     2|892079237|
+------+------+------+---------+
only showing top 20 rows



Step 2: Build a recommendation model using Alternating Least Squares

In [0]:
# split training and testing
(training, test) = df2.randomSplit([0.8, 0.2])

In [0]:
# Build the recommendation model using ALS on the training data
als = ALS(userCol="userid", itemCol="itemid", ratingCol="rating", nonnegative=True)
model = als.fit(training)

In [14]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Model results in RMSE = nan due to cold start problem

Root-mean-square error = nan


RMSE for first model is nan due to cold start problem

Step 4 : Resolving Cold start problem and improving performance by cross validation

Resolving cold start problem

In [37]:
# Build new recommendation model using ALS on the training data resolving cold start problem
als_basic = ALS(userCol="userid", itemCol="itemid", ratingCol="rating",coldStartStrategy="drop", nonnegative=True)
model_basic = als_basic.fit(training)
# Evaluate the model by computing the RMSE on the test data
predictions = model_basic.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Basic model gives RMSE value of 0.92

Root-mean-square error = 0.9203312857132404


RMSE for basic model after resolving cold start problem is 0.92

In [39]:
als = ALS(userCol="userid", itemCol="itemid", ratingCol="rating",coldStartStrategy="drop", nonnegative=True)

#Tuning model using ParamGridBuilder
param_grid=ParamGridBuilder()\
            .addGrid(als.rank,(12,13,14))\
            .addGrid(als.maxIter,(5,10,15))\
            .addGrid(als.regParam,[0.01,0.05,0.10])\
            .build()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
#CrossValidation
tvs=TrainValidationSplit(estimator=als,estimatorParamMaps=param_grid,evaluator=evaluator)
model = tvs.fit(training)          

best_model=model.bestModel
predictions = best_model.transform(test)

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.9162053255198508


Root-mean-square error = 0.916

Improving performance by cross validation

In [41]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

als = ALS(userCol="userid", itemCol="itemid", ratingCol="rating",coldStartStrategy="drop", nonnegative=True)

#Tuning model using ParamGridBuilder
param_grid=ParamGridBuilder()\
            .addGrid(als.rank,(3,5,8))\
            .addGrid(als.maxIter,(10,15,25))\
            .addGrid(als.regParam,[0.01,0.05,0.10])\
            .build()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
#CrossValidation
tvs=CrossValidator(estimator=als,estimatorParamMaps=param_grid,evaluator=evaluator,numFolds=5)
model = tvs.fit(training)          

best_model=model.bestModel
predictions = best_model.transform(test)

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))
print("Best Rank:",best_model.rank)
print("Best MaxIter:",best_model._java_obj.parent().getMaxIter())
print("Best RegParam:",best_model._java_obj.parent().getRegParam())

Root-mean-square error = 0.9147682466775412
Best Rank: 5
Best MaxIter: 25
Best RegParam: 0.1


Improved RMSE
Root-mean-square error = 0.914
Best Rank: 5
Best MaxIter: 25
Best RegParam: 0.1

Step 5 : Output top 10 movies for all the users and write output into file

In [0]:
import pandas as pd
Top10allusers=best_model.recommendForAllUsers(10)
Top10allusers=Top10allusers.toPandas()
user=[]
items=[]
for row in range(len(Top10allusers)):
  user.append(Top10allusers.iloc[row,0])
  rec=""
  for item in Top10allusers.iloc[row,1]:
    rec=rec+","+str(item.asDict()["itemid"])
  items.append(rec[2:])

Toprecdf=pd.DataFrame(data=zip(user,items),columns=["userid","items_recommended"])
Toprecdf.to_csv("Top10rec.txt", sep='\t')

In [43]:
#Displaying 10 rows of final result
Toprecdf.head(10)


,userid,items_recommended
0,471,"278,1429,1483,1443,1192,1159,1605,1598,1389,1400"
1,463,"449,1358,543,1512,1599,1642,947,1398,1643,814"
2,833,"368,1524,1367,1536,179,320,1643,1629,1405,1388"
3,496,"99,320,1240,1524,1282,1607,850,56,889,1005"
4,148,"14,867,408,1449,1599,1122,119,169,1398,483"
5,540,"449,867,1398,1636,1645,1650,1651,1122,119,814"
6,392,"449,814,1643,1599,119,867,1398,483,1358,318"
7,243,"449,1643,1642,1599,1398,1512,1193,814,543,1358"
8,623,"233,814,1599,1449,867,1064,626,318,1398,483"
9,737,"449,1524,1282,1643,119,1388,1368,320,56,1405"
